# [2021 Week 33 | Power BI: Create Magic Tables](http://www.workout-wednesday.com/pbi-2021-w33/)

>**Table of contents:**
>
>&ensp;&ensp;[Introduction](#Introduction) <br>
>&ensp;&ensp;[Instructions](#Instructions) <br>
>&ensp;&ensp;[Workings](#Workings) <br>
>&ensp;&ensp;[Results](#Results)


### Introduction
The customers who signed up a service need to make a deposit.
    
Their details, including the signup and deposit dates, are recorded in the [dataset](https://nikolailic-my.sharepoint.com/:x:/g/personal/nikola_nikolailic_onmicrosoft_com/EVsfgWvUVllBuTHaYk1s7jUBMCV5x4psl6KRfZAcjdLKXg?e=dHM31A).

The task is to create a table to track the customers' deposit info. For reference, below shows the table created using Microsoft Power BI, as provided in the [challenge's page](http://www.workout-wednesday.com/pbi-2021-w33/).

![image1](https://drive.google.com/uc?export=view&id=10BWRV3T1Y5X8n-lRf0xKWOSjqIDB9M5v)

### Instructions
- Include a title 'Signups VS Deposits'.


- Include the information as shown in the table header. These information are available in the dataset, except for the last two columms:
    
    - Signup
        - Contains the number of days between the signup date and today's date
        - Gradual background color based on the signup days
        - Enable for sorting
    
    - Deposit Info
        - Label the customers who made a deposit with ✔️
        - Label the customers who signed up 2 days ago, but haven’t made a deposit yet with ❗
        - Label the customers who signed up more than 2 days ago and still haven’t made a deposit with ❌


- Enable filter based on the number of days since the customer signed up.


- Enable filter based on the number of days between signup date and deposit date.


- Format the table to look like the reference table.

### Workings

In [1]:
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

In [2]:
# read the data
df = pd.read_csv('https://raw.githubusercontent.com/ywjet/Data-Visualization/main/Data/2021%20Week%2033_Power%20BI_Create%20Magic%20Tables.csv')
df.head()

,Customer,City,State,Country,Referrer ID,Signup Date,First Deposit
0,Jon Yang,Alexandria,New South Wales,Australia,SO48520,09/06/2021,10/06/2021
1,Eugene Huang,Coffs Harbour,New South Wales,Australia,SO48521,08/06/2021,13/06/2021
2,Ruben Torres,Darlinghurst,New South Wales,Australia,SO48522,13/06/2021,NaN
3,Christy Zhu,Goulburn,New South Wales,Australia,SO48523,18/07/2021,20/07/2021
4,Elizabeth Johnson,Lane Cove,New South Wales,Australia,SO48524,19/07/2021,20/07/2021


In [3]:
df['First Deposit'].isna().sum()

8

The null values in the 'First Deposit' column indicate the customers who have not make a deposit, and there are 8 of them.
<br>
Let's see if other columns have any null value.

In [4]:
df.isna().any()

Customer         False
City             False
State            False
Country          False
Referrer ID      False
Signup Date      False
First Deposit     True
dtype: bool

Here, only the 'First Deposit' column contains null values.

Before doing some calculations, we'll check if the last two columns are datetimes.

In [5]:
df.dtypes

Customer         object
City             object
State            object
Country          object
Referrer ID      object
Signup Date      object
First Deposit    object
dtype: object

The data in the last two columns are strings and we need to convert them to datetimes.

In [6]:
df['Signup Date'] = pd.to_datetime(df['Signup Date'], dayfirst=True)
df['First Deposit'] = pd.to_datetime(df['First Deposit'], dayfirst=True)
df.dtypes

Customer                 object
City                     object
State                    object
Country                  object
Referrer ID              object
Signup Date      datetime64[ns]
First Deposit    datetime64[ns]
dtype: object

In [7]:
df.head()

,Customer,City,State,Country,Referrer ID,Signup Date,First Deposit
0,Jon Yang,Alexandria,New South Wales,Australia,SO48520,2021-06-09,2021-06-10
1,Eugene Huang,Coffs Harbour,New South Wales,Australia,SO48521,2021-06-08,2021-06-13
2,Ruben Torres,Darlinghurst,New South Wales,Australia,SO48522,2021-06-13,NaT
3,Christy Zhu,Goulburn,New South Wales,Australia,SO48523,2021-07-18,2021-07-20
4,Elizabeth Johnson,Lane Cove,New South Wales,Australia,SO48524,2021-07-19,2021-07-20


In [8]:
# the number of days since the customer signed up
df['Signup Days'] = (pd.Timestamp.today() - df['Signup Date']).dt.days
# the number of days between signup date and deposit date
df['Deposit Days'] = (df['First Deposit'] - df['Signup Date']).dt.days

# create the Signup column
df['Signup'] = df['Signup Days'].astype(str)
for i, day in enumerate(df['Signup']):
    if int(day) > 1:
        df['Signup'][i] = day + ' days ago'
    else:
        df['Signup'][i] = day + ' day ago'    

# create the Deposit Info column
df['Deposit Info'] = 1
for i, deposit in enumerate(df['First Deposit']):
    if pd.isna(deposit):
        signup = int(df['Signup'][i].split()[0])
        if signup > 2:
            df['Deposit Info'][i] = '❌'
        else:
            df['Deposit Info'][i] = '❗'
    else:
        df['Deposit Info'][i] = '✔️'

df.head()

,Customer,City,State,Country,Referrer ID,Signup Date,First Deposit,Signup Days,Deposit Days,Signup,Deposit Info
0,Jon Yang,Alexandria,New South Wales,Australia,SO48520,2021-06-09,2021-06-10,98,1.0,98 days ago,✔️
1,Eugene Huang,Coffs Harbour,New South Wales,Australia,SO48521,2021-06-08,2021-06-13,99,5.0,99 days ago,✔️
2,Ruben Torres,Darlinghurst,New South Wales,Australia,SO48522,2021-06-13,NaT,94,NaN,94 days ago,❌
3,Christy Zhu,Goulburn,New South Wales,Australia,SO48523,2021-07-18,2021-07-20,59,2.0,59 days ago,✔️
4,Elizabeth Johnson,Lane Cove,New South Wales,Australia,SO48524,2021-07-19,2021-07-20,58,1.0,58 days ago,✔️


The null values in the 'Deposit Days' column represent the customers who has not make a deposit. <br>
We'll convert the null values to 0.

In [9]:
df['Deposit Days'].fillna(0, inplace=True)
df.head()

,Customer,City,State,Country,Referrer ID,Signup Date,First Deposit,Signup Days,Deposit Days,Signup,Deposit Info
0,Jon Yang,Alexandria,New South Wales,Australia,SO48520,2021-06-09,2021-06-10,98,1.0,98 days ago,✔️
1,Eugene Huang,Coffs Harbour,New South Wales,Australia,SO48521,2021-06-08,2021-06-13,99,5.0,99 days ago,✔️
2,Ruben Torres,Darlinghurst,New South Wales,Australia,SO48522,2021-06-13,NaT,94,0.0,94 days ago,❌
3,Christy Zhu,Goulburn,New South Wales,Australia,SO48523,2021-07-18,2021-07-20,59,2.0,59 days ago,✔️
4,Elizabeth Johnson,Lane Cove,New South Wales,Australia,SO48524,2021-07-19,2021-07-20,58,1.0,58 days ago,✔️


The 'Deposit Info' column actually contains ✔️ and ❌ only. <br>
No ❗ is included because by now, all customers have been signed up for more than 2 days.

In [10]:
df['Deposit Info'].unique()

array(['✔️', '❌'], dtype=object)

The data is now ready and we'll start creating the table.

In [11]:
# columns to be included in the Dash table
col = ['Customer', 'City', 'State', 'Country', 'Referrer ID', 'Signup', 'Deposit Info']

In [12]:
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output
import dash_table

Below is the code to create the gradual background colour for the 'Signup' column. <br>
The code is available in the [documentation](https://dash.plotly.com/datatable/conditional-formatting) under the title 'Highlighting cells by value with a colorscale like a heatmap'. <br>
Some modifications are done so that the conditional format can be applied to categorical 'Signup' column.

In [13]:
# parameters to be used in the filter later
maxSD = df['Signup Days'].max()
maxDD = df['Deposit Days'].max()

# compute the color scale for the signup columns
import colorlover

n_bins=5
bounds = [i * (1.0 / n_bins) for i in range(n_bins + 1)]

df_numeric_columns = df[['Signup Days']]
df_max = df_numeric_columns.max().max()
df_min = df_numeric_columns.min().min()
ranges = [
    ((df_max - df_min) * i) + df_min
    for i in bounds
]
styles = []
for i in range(1, len(bounds)):
    min_bound = ranges[i - 1]
    max_bound = ranges[i]
    backgroundColor = colorlover.scales[str(n_bins)]['seq']['Blues'][i - 1]
    color = 'white' if i > len(bounds) / 2. else 'inherit'

    for column in df_numeric_columns:
        styles.append({
            'if': {
                'filter_query': (
                    '{{{column}}} >= {min_bound}' +
                    (' && {{{column}}} < {max_bound}' if (i < len(bounds) - 1) else '')
                ).format(column=column, min_bound=min_bound, max_bound=max_bound),
                'column_id': 'Signup'
            },
            'backgroundColor': backgroundColor,
            'color': color
        })

In [14]:
app = dash.Dash(__name__)

app.layout = html.Div([
    html.H1('Signups VS Deposits', style={'text-align':'center'}),
    
    html.Br(),
    
    ################# Signup Filter ##################################
    
    html.P('Number of day(s) since the customer signed up:'),
    
    dcc.Input(
        id='start1',
        type='number',
        min=0,
        max=maxSD,
        value=0,
        style={'width':'10%'}
    ),
    
    dcc.Input(
        id='end1',
        type='number',
        min=0,
        value=maxSD,
        style={
            'width':'10%',
            'margin-left':'1px',
            'margin-right':'15px'
        }
    ),
    
    html.I(id='description1'),
    
    html.Br(),
    
    html.Br(),
    
    ################# Deposit Filter ##################################
    
    html.P('Number of day(s) between signup and deposit:'),

    dcc.Input(
        id='start2',
        type='number',
        min=0,
        max=maxDD,
        value=0,
        style={'width':'10%'}
    ),
    
    dcc.Input(
        id='end2',
        type='number',
        min=0,
        value=maxDD,
        style={
            'width':'10%',
            'margin-left':'1px',
            'margin-right':'15px'
        }
    ),
    
    html.I(id='description2'),    
    
    html.Br(),
    
    html.Br(),
    
    ################# Sorting Options ##############################
    
    html.P('Sort Signup order:'),
    
    dcc.RadioItems(
        id='SortOpt',
        options=[
            {'label': 'Ascending', 'value': 'asc'},
            {'label': 'Descending', 'value': 'des'}
        ],
        value='asc',
        
        inputStyle={"margin-left": "10px"}
    ),
    
    html.Br(),
    
    html.Br(),
    
    ################# Data Table ##################################
    
    dash_table.DataTable(
        id='table',
        columns=[{"name": i, "id": i} 
                 for i in col],
        data=df.to_dict('records'),
        style_cell=dict(textAlign='left'),
        style_header=dict(
            backgroundColor='darkblue',
            color='white'
        ),
        style_data={
            'border-top':'1px solid blue',
            'border-bottom':'1px solid blue'
        },
        style_data_conditional=[
            {
                'if': {'column_id': 'Customer'},
                'color':'mediumblue'
            },
            {
                'if': {'column_id': 'City'},
                'color':'royalblue'
            },
            {
                'if': {'column_id': 'Country'},
                'color':'mediumblue'
            },
            {
                'if': {'column_id': 'Customer'},
                'border-left':'1px solid blue'
            },
            {
                'if': {'column_id': 'Deposit Info'},
                'border-right':'1px solid blue'
            }
        ] + styles
    ),
    
    html.Br()
])


# descriptions of the filters applied are included to avoid confusion
@app.callback(
    Output('description1', 'children'),
    Input('start1', 'value'),
    Input('end1', 'value')
)
def describe1(start, end):
    if start == end:
        return 'Viewing customer(s) who signed up {} day(s) ago.'.format(start)
    else:
        return 'Viewing customer(s) who signed up between {} and {} day(s) ago.'.format(start, end)


@app.callback(
    Output('description2', 'children'),
    Input('start2', 'value'),
    Input('end2', 'value')
)
def describe2(start, end):
    if (start==end) & (start==0):
        return 'Viewing customer(s) who have not make a deposit yet.'
    elif start==end:
        return 'Viewing customer(s) who deposited {} day(s) after signup.'.format(start)
    else:
        return 'Viewing customer(s) who deposited between {} and {} day(s) after signup.'.format(start, end)


@app.callback(
    Output('table', 'data'),
    Input('start1', 'value'),
    Input('end1', 'value'),
    Input('start2', 'value'),
    Input('end2', 'value'),
    Input('SortOpt', 'value')
)
def update_table(start1, end1, start2, end2, option):
    dff = df[(df['Signup Days'] >= start1) & (df['Signup Days'] <= end1)].copy()
    dff = dff[(dff['Deposit Days'] >= start2) & (dff['Deposit Days'] <= end2)]
    if option == 'asc':
        dff = dff.sort_values('Signup Days')
    else:
        dff = dff.sort_values('Signup Days', ascending=False)
    return dff.to_dict('records')

if __name__ == '__main__':
#     app.run_server(debug=True, use_reloader=False)
    app.run_server(debug=False)

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app '__main__' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)


### Results

![image2](https://drive.google.com/uc?export=view&id=1MlnulqtzoCRJj2-mFpGKUL9AXYWqfian)